<a href="https://colab.research.google.com/github/RB-Labs/HPC.CourseWork/blob/main/HPC_CourseWork_03_NaiveBayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

os.environ['APACHE_SPARK_DISTR_NAME'] = 'spark-3.0.2-bin-hadoop2.7'
os.environ['APACHE_SPARK_URL'] = 'https://downloads.apache.org/spark/spark-3.0.2/spark-3.0.2-bin-hadoop2.7.tgz'
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-8-openjdk-amd64'
os.environ['SPARK_HOME'] = '/content/spark-3.0.2-bin-hadoop2.7'

In [2]:
import os.path
from google.colab import files
if not os.path.isfile('kaggle.json'):
    files.upload()

Saving kaggle.json to kaggle.json


In [3]:
%%bash
apt-get update > /dev/null
apt-get purge -y openjdk-11* -qq > /dev/null && sudo apt-get autoremove -y -qq > /dev/null
apt-get install -y openjdk-8-jdk-headless -qq > /dev/null
pip install --upgrade -q pyspark==3.0.2 spark-nlp==3.0.3 findspark > /dev/null
[ ! -d $APACHE_SPARK_DISTR_NAME ] \
   && wget -q $APACHE_SPARK_URL \
   && tar xf $APACHE_SPARK_DISTR_NAME.tgz \
   && rm -f $APACHE_SPARK_DISTR_NAME.tgz
[ ! -d /root/.kaggle ] \
    && mkdir /root/.kaggle \
    && cp kaggle.json /root/.kaggle \
    && chmod 600 /root/.kaggle/kaggle.json
[ ! -f lenta-ru-news.csv ] \
    && kaggle datasets download -d yutkin/corpus-of-russian-news-articles-from-lenta \
    && unzip -q corpus-of-russian-news-articles-from-lenta.zip \
    && rm -f corpus-of-russian-news-articles-from-lenta.zip

100%|##########| 584M/584M [00:05<00:00, 104MB/s] 


In [4]:
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
from pyspark.ml.feature import HashingTF, IDF, StringIndexer, IndexToString
from pyspark.ml.classification import NaiveBayes
import pyspark.sql.functions as F
from sparknlp.annotator import *
from sparknlp.common import *
from sparknlp.base import *
import sparknlp

In [5]:
spark = sparknlp.start()

In [6]:
main_df = spark.read.csv(
    'lenta-ru-news.csv',
    header=True,
    multiLine=True,
    escape="\"")
main_df.count()

800975

In [7]:
df = main_df \
    .na.drop(subset=["topic"]) \
    .select(["text", "topic"]) \
    .limit(200000) \
    .withColumn("text", F.regexp_replace(F.col("text"), "[\n\r]", " ")) \
    .withColumn("text", F.regexp_replace(F.col("text"), ".Rambler Title ", "")) \
    .withColumn("text", F.trim(F.col("text")))

In [8]:
df.show()

+--------------------+----------+
|                text|     topic|
+--------------------+----------+
|Бои у Сопоцкина и...|Библиотека|
|Министерство наро...|Библиотека|
|Штабс-капитан П. ...|Библиотека|
|Фотограф-корреспо...|Библиотека|
|Лица, приехавшие ...|Библиотека|
|Как стало известн...|    Россия|
|В зале игровых ав...|    Россия|
|Япония приняла ре...|    Россия|
|Британцы отмечают...|       Мир|
|В понедельник дир...|    Россия|
|С 1 сентября на в...|    Россия|
|Указом президента...|    Россия|
|Сегодня областной...|    Россия|
|Бывший шеф Службы...|    Россия|
|подземный толчок ...|       Мир|
|Сегодня утром в р...|    Россия|
|Намеченная на сег...|    Россия|
|На состоявшейся с...|    Россия|
|15 представителей...|       Мир|
|На юге Киргизии, ...|       Мир|
+--------------------+----------+
only showing top 20 rows



In [9]:
df_count = df.groupBy("topic").count()

In [10]:
df_count.count()

12

In [11]:
df_count.show()

+-----------------+-----+
|            topic|count|
+-----------------+-----+
|       Библиотека|    5|
|           Россия|62927|
|              Мир|52103|
|        Экономика|20782|
|   Интернет и СМИ|11591|
|            Спорт|14682|
|         Культура|12920|
|         Из жизни| 6537|
|Силовые структуры|    7|
|  Наука и техника| 7872|
|      Бывший СССР| 8881|
|              Дом| 1693|
+-----------------+-----+



In [12]:
train_df, test_df = df.randomSplit([0.8, 0.2], seed = 1)

In [13]:
document_assembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

In [14]:
sentence_detector = SentenceDetector()\
    .setInputCols(['document'])\
    .setOutputCol('sentence')

In [15]:
tokenizer = Tokenizer()\
    .setInputCols(['sentence'])\
    .setOutputCol('token')

In [16]:
stop_words_cleaner = StopWordsCleaner\
    .pretrained('stopwords_ru', 'ru')\
    .setInputCols(["token"]) \
    .setOutputCol("cleanTokens") \
    .setCaseSensitive(False)

stopwords_ru download started this may take some time.
Approximate size to download 2.9 KB
[OK!]


In [17]:
lemmatizer = LemmatizerModel\
    .pretrained("lemma", "ru") \
    .setInputCols(["cleanTokens"]) \
    .setOutputCol("lemma")

lemma download started this may take some time.
Approximate size to download 1.3 MB
[OK!]


In [18]:
finisher = Finisher() \
    .setInputCols(["lemma"]) \
    .setOutputCols(["token_features"]) \
    .setOutputAsArray(True) \
    .setCleanAnnotations(False)

In [19]:
hashing_TF = HashingTF(
    inputCol="token_features",
    outputCol="raw_features")

In [20]:
idf = IDF(
    inputCol="raw_features",
    outputCol="features",
    minDocFreq=5)

In [21]:
topic_indexer = StringIndexer(
    inputCol="topic",
    outputCol="label")

In [22]:
bayes_classificator = NaiveBayes(
    smoothing=111)

In [23]:
topic_to_string_indexer = IndexToString(
    inputCol="label",
    outputCol="article_class")

In [24]:
pipeline = Pipeline(
    stages=[
        document_assembler,
        sentence_detector,
        tokenizer,
        stop_words_cleaner,
        lemmatizer,
        finisher,
        hashing_TF,
        idf,
        topic_indexer,
        bayes_classificator,
        topic_to_string_indexer])

In [25]:
%%time
classification_model = pipeline.fit(train_df)

CPU times: user 4.93 s, sys: 596 ms, total: 5.52 s
Wall time: 31min 45s


In [26]:
from sklearn.metrics import classification_report, accuracy_score

In [32]:
%%time
df_bayes = classification_model \
    .transform(test_df) \
    .select("topic", "label", "prediction", "text")
df_bayes_pandas = df_bayes.toPandas()

CPU times: user 2 s, sys: 348 ms, total: 2.35 s
Wall time: 4min 29s


In [28]:
print(classification_report(df_bayes_pandas.label, df_bayes_pandas.prediction))

              precision    recall  f1-score   support

         0.0       0.64      0.90      0.75     12569
         1.0       0.65      0.84      0.73     10262
         2.0       0.83      0.69      0.75      4103
         3.0       0.99      0.86      0.92      3049
         4.0       0.90      0.67      0.77      2649
         5.0       0.88      0.30      0.44      2403
         6.0       0.95      0.04      0.08      1739
         7.0       0.96      0.23      0.37      1575
         8.0       0.50      0.00      0.00      1327
         9.0       0.00      0.00      0.00       353
        10.0       0.00      0.00      0.00         1
        11.0       0.00      0.00      0.00         3

    accuracy                           0.71     40033
   macro avg       0.61      0.38      0.40     40033
weighted avg       0.74      0.71      0.67     40033



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [29]:
print(accuracy_score(df_bayes_pandas.label, df_bayes_pandas.prediction))

0.7058177003971723


In [30]:
%%time
validation_df = df \
    .sample(withReplacement=False, fraction=0.5, seed=1) \
    .limit(20)
validation_df.show(truncate=40)

+----------------------------------------+----------+
|                                    text|     topic|
+----------------------------------------+----------+
|Штабс-капитан П. Н. Нестеров на днях,...|Библиотека|
|Фотограф-корреспондент Daily Mirror р...|Библиотека|
|В зале игровых автоматов в третьем яр...|    Россия|
|Япония приняла решение разморозить кр...|    Россия|
|Британцы отмечают сегодня скорбную да...|       Мир|
|Указом президента России Бориса Ельци...|    Россия|
|Сегодня областной центр Сахалина и Ку...|    Россия|
|15 представителей национал-большевист...|       Мир|
|Россия крайне негативно оценивает суд...|       Мир|
|По сведениям миссии ООН, передаваемым...|       Мир|
|Соединенные Штаты заплатили Китаю 4,5...|       Мир|
|"Женщины России" , возмущенные отсутс...|    Россия|
|Скандал по поводу "отмывания денег ру...|       Мир|
|Следствие по делу "Bank of  New York"...|       Мир|
|Трое заложников освобождены минувшей ...|       Мир|
|МИД РФ в распространенном в

In [31]:
%%time
validation_df_pred = classification_model \
    .transform(validation_df) \
    .select("topic", "label", "prediction", "text")
validation_df_pred.show(truncate=23)

+----------+-----+----------+-----------------------+
|     topic|label|prediction|                   text|
+----------+-----+----------+-----------------------+
|Библиотека| 11.0|       1.0|Штабс-капитан П. Н. ...|
|Библиотека| 11.0|       1.0|Фотограф-корреспонде...|
|    Россия|  0.0|       0.0|В зале игровых автом...|
|    Россия|  0.0|       2.0|Япония приняла решен...|
|       Мир|  1.0|       1.0|Британцы отмечают се...|
|    Россия|  0.0|       0.0|Указом президента Ро...|
|    Россия|  0.0|       0.0|Сегодня областной це...|
|       Мир|  1.0|       0.0|15 представителей на...|
|       Мир|  1.0|       1.0|Россия крайне негати...|
|       Мир|  1.0|       1.0|По сведениям миссии ...|
|       Мир|  1.0|       1.0|Соединенные Штаты за...|
|    Россия|  0.0|       0.0|"Женщины России" , в...|
|       Мир|  1.0|       0.0|Скандал по поводу "о...|
|       Мир|  1.0|       1.0|Следствие по делу "B...|
|       Мир|  1.0|       1.0|Трое заложников осво...|
|       Мир|  1.0|       1.0